Install dependencies

In [1]:
pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.

In [2]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Configure Gemini API Key

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


Import ADK components

In [4]:
from google.genai import types

from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


Helper functions

In [5]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers
import json
import re
from typing import Dict, Any, Optional, List


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


INTENT_PATTERNS = {
    "create": r"\b(create|make|add|open|start|book|schedule)\b",
    "update": r"\b(update|change|modify|edit|reschedule)\b",
    "delete": r"\b(delete|remove|cancel)\b",
    "retrieve": r"\b(get|show|find|lookup|search|list)\b",
    "summarise": r"\b(summarise|summarize|tl;dr|condense)\b",
}

print("✅ Helper functions defined.")

✅ Helper functions defined.


Configure Retry Options

In [6]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

JSON Prompt Agent

In [7]:
INSTRUCTION = """
You convert a script into a STRICT JSON object for a downstream Script Breakdown agent.

HARD RULES
- Return ONLY a JSON object. No Markdown fences, no headings, no prose, no backticks.
- Use double quotes for all keys/strings. No comments. No trailing commas.
- If you cannot infer a value, use null or [] (never omit required keys).
- Default scope.units to ["scenes","beats"] unless explicitly requested otherwise.
- If scene numbers are missing, set scope.assume_scene_numbers_if_missing to true.
- UK English for any free-text fields.

SCHEMA
{
  "schema_version": "1.1",
  "task": "script_breakdown_instruction",
  "project": {
    "title": "<string>",
    "series": "<string or null>",
    "episode": "<string or null>",
    "draft_version": "<string or null>"
  },
  "scope": {
    "units": ["scenes","beats"],
    "include": ["characters","locations","props","VFX","SFX","stunts"],
    "exclude": [],
    "assume_scene_numbers_if_missing": true
  },
  "delivery": {
    "outputs": ["scene_list","character_matrix","location_list","prop_list"],
    "format": "json",
    "granularity": "scene"
  },
  "rules": {
    "character_normalisation": "uppercase canonical names",
    "location_format": "INT or EXT then location then time",
    "prop_detection": "nouns with direct physical presence",
    "ambiguous_items": "flag with rationale",
    "spoilers": "allow"
  },
  "scenes": [
    {
      "page": <int or null>,
      "scene_number": "<string or null>",
      "slugline": "<string or null>",
      "description": "<one- or two-sentence scene description>",
      "interior_exterior": "INT|EXT|INT/EXT|null",
      "location": "<string or null>",
      "time_of_day": "DAY|NIGHT|null",
      "categories": ["VFX","SFX","Stunts","Crowds","Vehicles"],
      "characters": ["<UPPERCASE NAME>", "..."],
      "props": ["<prop>", "..."]
    }
  ],
  "notes": "<free-text guidance distilled from the script>",
  "input_excerpt_checksum": "<sha1 or placeholder>",
  "source_summary": {
    "page_count_estimate": "<int>",
    "approx_runtime_minutes": "<int or null>"
  }
}

DETECTION GUIDANCE
- TITLE: Use title page if present; otherwise derive from the first slugline; else "Untitled".
- SCENE LIST: Create one entry per scene-like unit (sluglines or obvious breaks).
- PAGE: If page numbers are not present, estimate or set null (do not invent precise numbers).
- SCENE NUMBER: Use the script’s number; if missing, set null (and rely on scope.assume_scene_numbers_if_missing=true).
- DESCRIPTION: 1–2 concise sentences summarising action/intent of the scene.
- INTERIOR/EXTERIOR: Parse from slugline (INT., EXT., or INT./EXT.); else null.
- LOCATION: From slugline; use canonical wording (e.g., "VILLA LOUNGE").
- TIME OF DAY: Map to DAY or NIGHT. If ambiguous terms (e.g., EVENING, DAWN, DUSK), choose DAY for daylight, NIGHT for darkness; if unclear, null.
- CATEGORIES: Include zero or more from the allowed list only:
  ["VFX","SFX","Stunts","Crowds","Vehicles"].
  Examples: explosions → "SFX", CG creatures → "VFX", staged fights → "Stunts", underwater shots → "Underwater".
- CHARACTERS: Uppercase canonical names (merge aliases).
- PROPS: Physical items interacted with or foregrounded (phones, envelopes, weapons, etc.).
- AMBIGUITY: If uncertain, use null and add a brief note/rationale in "notes".

RETURN ONLY THE JSON OBJECT.
""".strip()


json_prompt_agent = LlmAgent(
    name="json_prompt",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    description="Converts script text into structured JSON prompt that instructs script breakdown agent.",
    instruction=INSTRUCTION,
    tools=[],
    output_key="json_prompt",
)

print("✅ Json Prompt Agent defined.")


✅ Json Prompt Agent defined.


Custom Tool

In [8]:
def make_veo3_prompt(scene: dict) -> str:
    slug = scene.get("slugline", "") or ""
    desc = scene.get("description", "") or ""
    loc = scene.get("location", "") or ""
    tod = (scene.get("time_of_day", "") or "").upper()
    chars = scene.get("characters", []) or []

    characters_str = ", ".join(chars) if isinstance(chars, list) else str(chars)

    # 1) Cinematography
    # Simple rules based on INT / EXT and time of day
    interior_exterior = (scene.get("interior_exterior") or "").upper()
    if interior_exterior == "INT":
        base_shot = "Medium wide shot, steady camera, eye level framing"
    elif interior_exterior == "EXT":
        base_shot = "Wide establishing shot, slow drift, cinematic framing"
    else:
        base_shot = "Medium shot, stable framing"

    if tod == "NIGHT":
        base_shot += ", sensitive low light capture"
    elif tod == "DAY":
        base_shot += ", clean daylight exposure"

    cinematography = base_shot

    # 2) Subject
    subject = (
        f"The main focus is on the contestants and the Host in {loc}"
        if characters_str
        else f"The main focus is the environment at {loc}"
    )

    # 3) Action
    action = f"They are shown in the moment described as: {desc}"

    # 4) Context
    context_bits = []
    if slug:
        context_bits.append(f"Scene taken from script slugline {slug}")
    if tod:
        context_bits.append(f"Time of day is {tod.lower()}")
    if characters_str:
        context_bits.append(f"Key characters visible: {characters_str}")
    context = ". ".join(context_bits) if context_bits else ""

    # 5) Style and ambiance
    if tod == "NIGHT":
        style = (
            "Cinematic reality television look, soft practical lighting and pool or accent glows. "
            "Rich contrast, gentle depth of field, subtle noise, natural skin tones."
        )
    elif tod == "DAY":
        style = (
            "Cinematic reality television look with bright natural light, soft shadows and clean colours. "
            "High fidelity textures, gentle depth of field, balanced contrast."
        )
    else:
        style = (
            "Cinematic reality television look with balanced lighting, clear faces and natural colour. "
            "Smooth depth of field and filmic grading."
        )

    # Assemble final prompt
    parts = [
        cinematography,
        subject,
        action,
        context,
        style,
    ]
    # Filter out any empty segments and join into one sentence block
    prompt = ". ".join(p for p in parts if p).strip()
    if not prompt.endswith("."):
        prompt += "."

    return prompt


In [9]:
from openpyxl import Workbook
from typing import Dict, Any
import tempfile
import os

def get_script_breakdown_excel(json_prompt: Dict[str, Any]) -> dict:
    """Creates an Excel (.xlsx) script breakdown file from a JSON prompt.

    Args:
        json_prompt: JSON prompt dict containing a 'scenes' list.

    Returns:
        Success:
            {
                "status": "success",
                "file_path": "<absolute path to .xlsx file>"
            }
        Error:
            {"status": "error", "error_message": "<details>"}
    """

    try:
        scenes = json_prompt.get("scenes")
        if not isinstance(scenes, list):
            return {"status": "error", "error_message": "json_prompt.scenes must be a list"}

        headers = [
            "Page number",
            "Scene number",
            "Scene description",
            "Interior or exterior",
            "Location",
            "Time of day (day or night)",
            "Category (VFX, SFX, Vechicles, Stunts etc)",
            "Characters",
            "Veo 3 Visual Prompt",
        ]

        wb = Workbook()
        ws = wb.active
        ws.title = "Script Breakdown"

        # Header row
        ws.append(headers)

        # Populate rows
        for s in scenes:
            # Safe normalisation for categories
            categories_val = s.get("categories", [])
            if isinstance(categories_val, list):
                categories_str = ", ".join(str(v) for v in categories_val)
            else:
                categories_str = str(categories_val) if categories_val is not None else ""

            # Safe normalisation for characters
            characters_val = s.get("characters", [])
            if isinstance(characters_val, list):
                characters_str = ", ".join(str(v) for v in characters_val)
            else:
                characters_str = str(characters_val) if characters_val is not None else ""

            veo_prompt = make_veo3_prompt(s)

            row = [
                s.get("page", ""),
                s.get("scene_number", ""),
                s.get("description", ""),
                s.get("interior_exterior", ""),
                s.get("location", ""),
                s.get("time_of_day", ""),
                categories_str,
                characters_str,
                veo_prompt,
                
            ]
            ws.append(row)

        # Adjust column widths
        for col in ws.columns:
            max_length = max(len(str(cell.value)) if cell.value else 0 for cell in col)
            col_letter = col[0].column_letter
            ws.column_dimensions[col_letter].width = min(max_length + 2, 60)

        # Save directly in Kaggle working directory
        save_dir = "/kaggle/working"
        os.makedirs(save_dir, exist_ok=True)
        file_path = os.path.join(save_dir, "script_breakdown.xlsx")
        wb.save(file_path)

        return {"status": "success", "file_path": file_path}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


Script Breakdown Agent

In [10]:
script_breakdown_agent = LlmAgent(
    name="script_breakdown_agent",
    model=Gemini(
        model="gemini-2.5-flash",  # tools-capable
        retry_options=retry_config
    ),
    description="Uses JSON prompt and outputs an Excel breakdown.",
    instruction="""
    You are given this JSON script breakdown prompt:
    
    {json_prompt}
    
    Steps:
    1. Call get_script_breakdown_excel(json_prompt={json_prompt}).
    2. Return only the file path of the generated Excel document.
    """,
    tools=[get_script_breakdown_excel],
    output_key="breakdown_excel_path",
)

print("✅ Script Breakdown Agent defined.")

✅ Script Breakdown Agent defined.


Scheduling Agent

In [11]:
from typing import Optional, Dict
from datetime import datetime
from docx import Document
import os

def generate_production_schedule_doc(
    project_title: str,
    schedule_text: str,
    filename: Optional[str] = None,
) -> Dict[str, str]:
    """
    Generates a Word document (.docx) containing a production schedule and
    saves it directly into the Kaggle working directory (/kaggle/working/).
    """

    output_dir = "/kaggle/working"
    os.makedirs(output_dir, exist_ok=True)

    # Clean filename
    if filename is None:
        safe_title = "".join(c for c in project_title if c.isalnum() or c in (" ", "_", "-")).strip()
        safe_title = safe_title.replace(" ", "_")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{safe_title}_schedule_{timestamp}"

    docx_path = os.path.join(output_dir, f"{filename}.docx")

    # Create Word document
    doc = Document()
    doc.add_heading(project_title, level=0)
    doc.add_paragraph("Production Schedule", style="Intense Quote")
    doc.add_paragraph("")

    # Insert schedule lines
    for line in schedule_text.splitlines():
        stripped = line.strip()
        if stripped.endswith(":") and stripped.isupper():
            doc.add_heading(stripped, level=2)
        else:
            doc.add_paragraph(line)

    # Save the Word file
    doc.save(docx_path)

    return {
        "status": "success",
        "message": "Production schedule document created.",
        "file_path": docx_path,
    }


In [12]:
scheduling_agent = LlmAgent(

    name="SchedulingAgent",

    model=Gemini(

        model="gemini-2.5-flash",

        retry_options=retry_config

    ),

    instruction=""" Get outputs:  {breakdown_excel_path} 

    You are a scheduling agent and you will get findings and the results from the: {breakdown_excel_path} and create a detailed and world-class production 

    schedule. If you cannot find certain information or it is unclear just add "TBC". You make this as detailed as possible to make it straightforward for the user. You will follow this template: PROJECT INFORMATION
 
    Project Title:
     
    Client / Broadcaster:
     
    Production Company:
     
    Project Type: (e.g., TV series, film, digital content, promo, branded content)
     
    Version / Date:
     
    KEY CONTACTS
     
    Executive Producer:
     
    Producer:
     
    Production Manager:
     
    Director:
     
    Writer(s):
     
    Editor / Post Supervisor:
     
    Legal & Clearance:
     
    Other Departments: (e.g., Camera, Art, Sound, Graphics)
     
    PRODUCTION OVERVIEW
     
    Total Production Period:
     
    Shoot Location(s):
     
    Format: (e.g., 4K, HD, multicam, remote)
     
    Final Deliverables: (list the required output formats)
     
    MASTER PRODUCTION SCHEDULE
    
    1. Development
     
    Concept development:
     
    Research / fact-checking:
     
    Script outline:
     
    Script drafts & approvals:
     
    2. Pre-Production
     
    Hire crew:
     
    Cast / contributors confirmed:
     
    Locations scouted & locked:
     
    Permits / legal / insurance:
     
    Equipment booked:
     
    Schedule & call sheets created:
     
    Pre-production meeting (PPM):
     
    3. Production
     
    Shoot Day(s):
     
    Day #:
     
    Location:
     
    Crew Call:
     
    Wrap Time:
     
    Notes:
     
    4. Post-Production
     
    Media ingest:
     
    Rough cut v1:
     
    Rough cut review:
     
    Fine cut:
     
    Picture lock:
     
    Sound mix:
     
    Colour grade:
     
    Graphics & captions:
     
    Quality control (QC):
     
    5. Deliverables & Distribution
     
    Master export:
     
    Legal & compliance checks:
     
    Final delivery to client/broadcaster:
     
    Archive media:
     
    RISK & CONTINGENCY NOTES
     
    (List any foreseeable issues and backup plans)
     
    ADDITIONAL NOTES
     
    (Any production notes the agent should know)
     
    You will then create a generated word documnent of the production schedule for the user to download
     
    """ ,
    
        tools=[generate_production_schedule_doc]
    
    )
    
print("✅ Scheduling Agent created.")
 


 

✅ Scheduling Agent created.


Sequential Agent

In [13]:
#Sequential pipeline: Script text -> JSON -> Excel

sequential_agent = SequentialAgent(
    name="ScriptBreakdownPipeline",
    sub_agents=[json_prompt_agent, script_breakdown_agent, scheduling_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [14]:
#define runner
runner = InMemoryRunner(agent=sequential_agent)

# User input is the script text
response = await runner.run_debug(
    """<SCRIPT>
Title: The Game Within
Genre: Reality Drama / Competition
Episode: 01 - "First Impressions"
Page: 1
Logline: Twelve strangers enter a luxury villa to compete for fame, fortune, and trust. But when
alliances form and secrets leak, even friendship becomes a weapon.
SCENE 1 - ARRIVAL (EXT. LUXURY VILLA - DAY)
CAMERA: Drone sweeping across the ocean, revealing a sleek modern villa perched on a cliff.
Tropical music fades under VO.
NARRATOR (V.O.): Twelve strangers. One villa. And a game where every friendship could be your
downfall.
CUT TO:
One by one, contestants arrive. Each greeted by the HOST, charismatic and slightly mischievous.
HOST: (smiling to camera) Welcome to The Game Within. Over the next six weeks, these players
will compete for one hundred thousand dollars... and something far more valuable: trust.
MONTAGE: Quick cuts of contestants laughing, shaking hands, sizing each other up.
CONTESTANT 1 (JENNA): I’m not here to make friends... unless it helps me win.
CONTESTANT 2 (MARCUS): People underestimate nice guys. That’s their first mistake.
SCENE 2 - THE FIRST CHALLENGE (INT. VILLA LOUNGE - AFTERNOON)
SET: Contestants gathered in a circle of plush chairs. A large screen flickers to life.
ON SCREEN: The word TRUST flashes in bold red.
HOST (V.O.): Your first challenge is simple: tell a lie... and make everyone believe it’s true.
REACTIONS: Shocked laughter. Whispers. Nervous glances.
CONTESTANT 3 (LUCAS): Wait... so we start by lying?
HOST: (appearing via screen) Exactly. The player who convinces the most people wins immunity
tonight.
CUT TO CONFESSIONALS:
JENNA (to camera): I’m a trained actress. This is my arena.
MARCUS: I hate lying. But losing? I hate that more.
SCENE 3 - NIGHTFALL / STRATEGY (EXT. POOL AREA - NIGHT)
CAMERA: Handheld shots. Dimly lit with fairy lights and reflections from the pool.
Small groups form, whispers float over cocktails.
LUCAS: (quietly to AMY) You think Jenna’s bluffing? She’s too confident.
AMY: Confidence is her weapon. You should be more worried about Marcus — he’s pretending to
be naive.
CUT TO: MARCUS sits alone, gazing at the water. The sound of waves builds as we hear his
voiceover.
MARCUS (V.O.): If they think I’m harmless, I’ve already won.
FADE OUT.
SCENE 4 - ELIMINATION ROOM (INT. CEREMONIAL CHAMBER - NIGHT)
SET DESIGN: A glowing round table. Ambient blue lights. Contestants’ faces tense.
HOST: Tonight, one of you will leave the villa. Remember — in The Game Within, lies have
consequences.
CUT TO: Contestants exchange suspicious looks.
HOST (CONT'D): The player with the fewest votes of trust... is LUCAS.
LUCAS: (shocked) You’ve got to be kidding.
JENNA (V.O.): One down. Eleven to go.
FADE OUT.
SCENE 5 - TEASER (EXT. VILLA - DAWN)
MONTAGE: Quick flashes — tears, confrontation, a hidden envelope.
NARRATOR (V.O.): Next time on The Game Within... the truth gets personal.
FADE TO BLACK.
END OF SAMPLE SCRIPT
</SCRIPT>
""" 
)

print("✅ Script breakdown excel created")
print("✅ Schedule document created")



 ### Created new session: debug_session_id

User > <SCRIPT>
Title: The Game Within
Genre: Reality Drama / Competition
Episode: 01 - "First Impressions"
Page: 1
Logline: Twelve strangers enter a luxury villa to compete for fame, fortune, and trust. But when
alliances form and secrets leak, even friendship becomes a weapon.
SCENE 1 - ARRIVAL (EXT. LUXURY VILLA - DAY)
CAMERA: Drone sweeping across the ocean, revealing a sleek modern villa perched on a cliff.
Tropical music fades under VO.
NARRATOR (V.O.): Twelve strangers. One villa. And a game where every friendship could be your
downfall.
CUT TO:
One by one, contestants arrive. Each greeted by the HOST, charismatic and slightly mischievous.
HOST: (smiling to camera) Welcome to The Game Within. Over the next six weeks, these players
will compete for one hundred thousand dollars... and something far more valuable: trust.
MONTAGE: Quick cuts of contestants laughing, shaking hands, sizing each other up.
CONTESTANT 1 (JENNA): I’m not here t

script_breakdown_agent > /kaggle/working/script_breakdown.xlsx


SchedulingAgent > PROJECT INFORMATION
 
Project Title: The Game Within
 
Client / Broadcaster: TBC
 
Production Company: TBC
 
Project Type: Reality Drama / Competition
 
Version / Date: Episode: 01 - "First Impressions" / TBC
 
KEY CONTACTS
 
Executive Producer: TBC
 
Producer: TBC
 
Production Manager: TBC
 
Director: TBC
 
Writer(s): TBC
 
Editor / Post Supervisor: TBC
 
Legal & Clearance: TBC
 
Other Departments: (e.g., Camera, Art, Sound, Graphics) TBC
 
PRODUCTION OVERVIEW
 
Total Production Period: TBC (Script implies a six-week in-story competition, but actual production period for Episode 01 is TBC)
 
Shoot Location(s): Luxury Villa (Exterior and Interior), Villa Lounge (Interior), Pool Area (Exterior), Ceremonial Chamber (Interior)
 
Format: TBC (Script indicates drone camera for establishing shots, handheld for strategy scenes, and on-screen graphics.)
 
Final Deliverables: TBC (e.g., Master video files, broadcast-ready formats, social media cut-downs)
 
MASTER PRODUCTION SC